In [40]:
import tonic
import tonic.transforms as transforms
import numpy as np
#import tensorflow as tf

transformer= transforms.ToFrame(sensor_size = (1280, 720), time_window = 100000)
transformer = transforms.Denoise(filter_time= 1000)

In [41]:
#path = "/Volumes/CVI2_SPACE_1/SPADES"
path= "/home/brini/hackathon/spades/SPADES"

In [46]:
import tonic
import os
import pandas as pd
import numpy as np

class SpacecraftEventDataset(tonic.Dataset):
    def __init__(self, root_dir, dataset_type="synthetic", transform = transformer):
        """
        Args:
            root_dir (str): Root directory containing the synthetic or Real dataset folders.
            dataset_type (str): Type of dataset to load ("Synthetic" or "Real").
            transform (callable, optional): Optional transform to apply to the events.
        """
    
        self.root_dir = root_dir
        self.dataset_type = dataset_type
        self.samples = self._load_samples()
        self.transform= transform

    def _load_samples(self):
        """
        Load event and label files and pair them appropriately.
        
        Returns:
            list of tuples: Each tuple contains paths to an event file and its corresponding label file (for synthetic data).
        """
        samples = []
        events_dir = os.path.join(self.root_dir, self.dataset_type, "events")
        
        if self.dataset_type == "synthetic":
            labels_dir = os.path.join(self.root_dir, self.dataset_type, "labels")
            for file_name in sorted(os.listdir(events_dir)):
                if file_name.endswith(".csv"):
                    event_file = os.path.join(events_dir, file_name)
                    label_file = os.path.join(labels_dir, file_name)
                    samples.append((event_file, label_file))
        elif self.dataset_type == "Real":
            for file_name in sorted(os.listdir(events_dir)):
                if file_name.endswith(".csv"):
                    event_file = os.path.join(events_dir, file_name)
                    samples.append((event_file, None))
        return samples

    def _load_events(self, file_path, is_synthetic=True):
        """
        Load event data from a CSV file.
        
        Args:
            file_path (str): Path to the event CSV file.
            is_synthetic (bool): Whether the event data is synthetic or real.

        Returns:
            np.ndarray: Array of events with columns [timestamp, x, y, polarity].
        """
        df = pd.read_csv(file_path, header=None, names=['timestamp', 'x', 'y', 'polarity'])
        events = df[['timestamp', 'x', 'y', 'polarity']].values
        return events

    def _load_labels(self, file_path):
        """
        Load label data from a CSV file.
        
        Args:
            file_path (str): Path to the label CSV file.

        Returns:
            np.ndarray: Array of labels with columns [Tx, Ty, Tz, Qx, Qy, Qz, Qw].
        """
        df = pd.read_csv(file_path)
        labels = df[['Tx', 'Ty', 'Tz', 'Qx', 'Qy', 'Qz', 'Qw']].values
        return labels

    def __getitem__(self, index):
        """
        Get a sample and its label (if available).

        Args:
            index (int): Index of the sample.

        Returns:
            tuple: (events, labels) where events is an array of event data, and labels is an array of labels (if synthetic data).
        """
        event_file, label_file = self.samples[index]
        is_synthetic = self.dataset_type == "synthetic"
        
        events = self._load_events(event_file, is_synthetic=is_synthetic)
        if label_file:
            labels = self._load_labels(label_file)
        else:
            labels = None
            
        print(f"Transform: {self.transform}")
        #if self.transform:
        print(events.shape)
        events = self.transform(events)

        return events, labels

    def __len__(self):
        return len(self.samples)

In [47]:
dataset = SpacecraftEventDataset(root_dir=path, transform=transformer)

In [48]:
len(dataset)

300

In [49]:
events,labels= dataset[1]

Transform: Denoise(filter_time=1000)
(9930299, 4)


TypeError: argument of type 'NoneType' is not iterable

In [36]:
print(events[1,:])
print(labels[1,:])

[0 544 290 True]
[-0.0446729   0.25294711  3.49160836  0.7431194   0.09160177 -0.6433912
  0.15946924]


In [ ]:
#import sys
#!{sys.executable} -m pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.3 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.9 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 7.2 MB/s eta 0:00:00a 0:00:01


In [40]:
import tensorflow as tf
keras_dataset = tf.data.Dataset.from_generator

2024-11-12 12:42:49.025994: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 12:42:49.033481: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 12:42:49.046374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731411769.064189    9121 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731411769.069305    9121 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 12:42:49.087199: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
keras_dataset = tf.data.Dataset.from_generator(
lambda: dataset_generator(dataset),output_signature=(
        tf.TensorSpec(shape=(128, 128, 10), dtype=tf.float32),  # Adjust shape as per your data
        tf.TensorSpec(shape=(), dtype=tf.int32)                 # Example: single integer label
        ))
keras_dataset = keras_dataset.batch(32).shuffle(buffer_size=100)

W0000 00:00:1731412022.605542    9121 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [31]:
print("Transform object:", transformer)


Transform object: ToFrame(sensor_size=(1280, 720), time_window=100000, event_count=None, n_time_bins=None, n_event_bins=None, overlap=0, include_incomplete=False, start_time=None, end_time=None)
